<a href="https://colab.research.google.com/github/simulate111/Production-Optimization/blob/main/01_01_Getting_Started_with_Pyomo_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Pyomo on Google Colab

Keywords: installation

This note notebook shows how to install the basic pyomo package on Google Colab, and then demonstrates the subsequent installation and use of various solvers including

* GLPK
* COIN-OR CBC
* COIN-OR Ipopt
* COIN-OR Bonmin
* COIN-OR Couenne
* COIN-OR Gecode

## Basic installation of Pyomo

We'll do a quiet installation of pyomo using `pip`.  This needs to be done once at the start of each Colab session.

In [1]:
!pip install -q pyomo

The installation of pyomo can be verified by entering a simple model. We'll use the model again in subsequent cells to demonstrate the installation and execution of various solvers.

In [2]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(expr = 40*model.x + 30*model.y, sense=maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x + 30*y

3 Constraint Declarations
    demand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :  40.0 :   True
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : x + y :  80.0 :   True
    laborB : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y : 100.0 :   True

6 Declarations: x y profit demand laborA laborB


## GLPK installation

Keywords: GLPK

[GLPK](https://en.wikibooks.org/wiki/GLPK) is a the open-source **G**NU **L**inear **P**rogramming **K**it available for use under the GNU General Public License 3. GLPK is a single-threaded simplex solver generally suited to small to medium scale linear-integer programming problems. It is written in C with minimal dependencies and is therefore highly portable among computers and operating systems. GLPK is often 'good enough' for many examples. For larger problems users should consider higher-performance solvers, such as COIN-OR CBC, that can take advantage of multi-threaded processors.

In [3]:
!apt-get install -y -qq glpk-utils

In [4]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0022728443145751953
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

## COIN-OR CBC installation

Keywords: cbc installation

[COIN-OR CBC](https://github.com/coin-or/Cbc) is a multi-threaded open-source **C**oin-or **b**ranch and **c**ut mixed-integer linear programming solver written in C++ under the Eclipse Public License (EPL). CBC is generally a good choice for a general purpose MILP solver for medium to large scale problems.

In [5]:
!apt-get install -y -qq coinor-cbc

In [6]:
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of nonzeros: 2
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created s

## COIN-OR Ipopt installation

Keywords: Ipopt installation

[COIN-OR Ipopt](https://github.com/coin-or/Ipopt) is an open-source **I**nterior **P**oint **Opt**imizer for large-scale nonlinear optimization available under the Eclipse Public License (EPL). It is well-suited to solving nonlinear programming problems without integer or binary constraints.

In [7]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "gurobi", "xpress", "cplex", "copt", "mosek", "coin"], # ipopt in coin # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="c64f519f-6d1d-4e0a-9823-df5ae92c7f05") # your license UUID (e.g., free ampl.com/ce or ampl.com/courses licenses)

Licensed to AMPL Academic License for <makhtari@abo.fi>.


In [8]:
#%pip install amplpy[pyomo]

In [9]:
from pyomo.environ import *

solver = SolverFactory("ipopt")
results = solver.solve(model, tee=True)


Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

In [10]:
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64

In [11]:
#SolverFactory('ipopt', executable='/content/ipopt').solve(model).write()
SolverFactory('ipopt').solve(model).write()


# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -.inf
  Upper bound: .inf
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.009680747985839844
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Pro

## COIN-OR Bonmin installation

[COIN-OR Bonmin](https://www.coin-or.org/Bonmin/Intro.html) is a **b**asic **o**pen-source solver for **n**onlinear **m**ixed-**in**teger programming problems (MINLP). It utilizes CBC and Ipopt for solving relaxed subproblems.

In [12]:
!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
!unzip -o -q bonmin-linux64

[bonmin-linux64.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of bonmin-linux64 or
        bonmin-linux64.zip, and cannot find bonmin-linux64.ZIP, period.


In [13]:
solver = SolverFactory("bonmin")
results = solver.solve(model, tee=True)

Bonmin 1.8.9 using Cbc 2.10.8 and Ipopt 3.12.13
bonmin: 
Cbc3007W No integer variables - nothing to do

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -2600        6 0.007078
Cbc3007W No integer variables - nothing to do

 	"Finished"


In [14]:
#SolverFactory('bonmin', executable='/content/bonmin').solve(model).write()
SolverFactory('bonmin').solve(model).write()

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -.inf
  Upper bound: .inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 0.011816740036010742
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit =  2600.0000259999

## COIN-OR Couenne installation

[COIN-OR Couenne](https://www.coin-or.org/Couenne/)  is attempts to find global optima for mixed-integer nonlinear programming problems (MINLP).

In [15]:
!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64

In [16]:
solver = SolverFactory("couenne")
results = solver.solve(model, tee=True)

Couenne 0.5.8 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: 
ANALYSIS TEST: Couenne: new cutoff value -2.6000000020e+03 (0.003104 seconds)
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -2600        6 0.007232
Couenne: new cutoff value -2.6000000260e+03 (0.011094 seconds)
Loaded instance "/tmp/tmpftwnnm_w.pyomo.nl"
Constraints:            3
Variables:              2 (0 integer)
Auxiliaries:            2 (0 integer)

Coin0506I Presolve 0 (-3) rows, 0 (-4) columns and 0 (-8) elements
Clp0000I Optimal - objective value -2600
Clp0032I Optimal objective -2600 - 0 iterations time 0.002, Presolve 0.00
Cbc3007W No integer variables - nothing to do
Clp0000I Optimal - objective value -2600

 	"Finished"

Linearization cuts added at root node:          3
Linearization cuts added in total:  

In [17]:
#SolverFactory('couenne', executable='/content/couenne').solve(model).write()
SolverFactory('couenne').solve(model).write()

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -.inf
  Upper bound: .inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: couenne\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 0.015735387802124023
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit =  2600.000025999

## Gecode installation

Keywords: Gecode installation

In [18]:
!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

In [19]:
ampl = ampl_notebook(
    modules=["gecode"], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="c64f519f-6d1d-4e0a-9823-df5ae92c7f05")

Licensed to AMPL Academic License for <makhtari@abo.fi>.


Gecode is a constraint programming (CP) solver.
It only supports:

Integer variables

Boolean variables

Finite domain discrete variables

❌ What Gecode cannot solve

Anything with:

Reals

NonNegativeReals

Continuous nonlinear expressions

Typical optimization models with continuous decision variables

Gecode solves constraint programming problems and does not support continuous variables. We therefore create a second model using exclusively discrete variables.

In [20]:
from pyomo.environ import *

# create a model
discrete_model = ConcreteModel()

# declare decision variables
discrete_model.x = Var(domain=NonNegativeIntegers)
discrete_model.y = Var(domain=NonNegativeIntegers)

# declare objective
discrete_model.profit = Objective(expr = 40*discrete_model.x + 30*discrete_model.y, sense=maximize)

# declare constraints
discrete_model.demand = Constraint(expr = discrete_model.x <= 40)
discrete_model.laborA = Constraint(expr = discrete_model.x + discrete_model.y <= 80)
discrete_model.laborB = Constraint(expr = 2*discrete_model.x + discrete_model.y <= 100)

discrete_model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x + 30*y

3 Constraint Declarations
    demand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :  40.0 :   True
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : x + y :  80.0 :   True
    laborB : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y : 100.0 :   True

6 Declarations: x y profit demand laborA laborB


In [21]:
solver = SolverFactory("gecode")
results = solver.solve(discrete_model, tee=True)

gecode 6.2.0: 

In [22]:
SolverFactory("gecode").available()


True

In [23]:
#SolverFactory('gecode', executable='/content/gecode').solve(discrete_model).write()
SolverFactory('gecode').solve(discrete_model).write()

# display solution
print('\nProfit = ', discrete_model.profit())

print('\nDecision Variables')
print('x = ', discrete_model.x())
print('y = ', discrete_model.y())

print('\nConstraints')
print('Demand  = ', discrete_model.demand())
print('Labor A = ', discrete_model.laborA())
print('Labor B = ', discrete_model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -.inf
  Upper bound: .inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: gecode 6.2.0\x3a optimal solution; 201 nodes, 0 fails, objective 2600
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.012328624725341797
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number o